In [0]:
!pip install fast-bert

!git clone https://github.com/NVIDIA/apex

!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-ds1j_4dl
Created temporary directory: /tmp/pip-req-tracker-fd2fz90_
Created requirements tracker '/tmp/pip-req-tracker-fd2fz90_'
Created temporary directory: /tmp/pip-install-11p05xex
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-xohr7t5n
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-fd2fz90_'
    Running setup.py (path:/tmp/pip-req-build-xohr7t5n/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info

     If your intention is to cross-compile, this is not an error.
    By default, Apex will cross-compile for Pascal (compute ca

In [0]:
!pip install pytorch_transformers

In [0]:
# Data handling and processing
import pandas as pd
import numpy as np
import os
from pathlib import Path

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

from fast_bert.data_cls import BertDataBunch
from pytorch_transformers import BertTokenizer
import torch

import collections
from tqdm import tqdm, trange
import sys
import random
#import apex
import datetime

from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
from fast_bert.prediction import BertClassificationPredictor

import logging

logger = logging.getLogger()

torch.cuda.empty_cache()

pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

import nltk
nltk.download('stopwords')
nltk.download('punkt')


#data_preprocessing
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup 
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression

##import dataset
dataset = pd.read_excel('train.xlsx')

##Fill missing values of Host with that link in the same row
for i in list(dataset[dataset['Host'].isnull() == True]['Host'].index):
    dataset.at[i,'Host'] = dataset.loc[i]['Link']
    
##Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(dataset[dataset['TRANS_CONV_TEXT'].isnull() == True].index):
    dataset.at[i, 'TRANS_CONV_TEXT'] = dataset.loc[i]['Title']
##Converting 'Date(ET)' object to datetime
dataset['Date(ET)'] = pd.to_datetime(dataset['Date(ET)'], infer_datetime_format=True)    

##Converting Datetime to timestamp
dataset['Date(ET)'] = dataset[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

##Dropping time(ET) and time(GMT)
dataset.drop(['Date(ET)', 'Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

##Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(dataset[dataset['Title'].isnull() == True]['Title'].index):
    dataset.at[i,'Title'] = dataset.loc[i]['TRANS_CONV_TEXT']

## TEST Data
test = pd.read_csv('test.csv', encoding = 'utf-8')

##Fill missing values of Host with that link in the same row
for i in list(test[test['Host'].isnull() == True]['Host'].index):
    test.at[i,'Host'] = test.loc[i]['Link']
    
#Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(test[test['TRANS_CONV_TEXT'].isnull() == True].index):
    test.at[i, 'TRANS_CONV_TEXT'] = test.loc[i]['Title']

test.at[441,'Date(ET)'] = test.loc[441, 'Time(ET)']

#Converting 'Date(ET)' object to datetime
test['Date(ET)'] = pd.to_datetime(test['Date(ET)'], infer_datetime_format=True)    

#Converting Datetime to timestamp
test['Date(ET)'] = test[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

#Dropping time(ET) and time(GMT)
test.drop(['Date(ET)','Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

#Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(test[test['Title'].isnull() == True]['Title'].index):
    test.at[i,'Title'] = test.loc[i]['TRANS_CONV_TEXT']

index = test['Index']
index = list(index)

test.drop(['Index'], axis = 1, inplace = True)

#feature_selection
y = dataset['Patient_Tag'].tolist()

dataset['Story'] = 'a'


for i in range(len(dataset)):
    dataset.at[i, 'Story'] = dataset['Source'][i] + ' ' + dataset['Host'][i] + ' ' + str(dataset['Link'][i]) + ' ' + dataset['Title'][i] + ' ' + dataset['TRANS_CONV_TEXT'][i]

dataset.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT'], axis = 1, inplace = True)


test['Story'] = 'a'


for i in range(len(test)):
    test.at[i, 'Story'] = test['Source'][i] + ' ' + test['Host'][i] + ' ' + str(test['Link'][i]) + ' ' + test['Title'][i] + ' ' + test['TRANS_CONV_TEXT'][i]

test.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT', 'Unnamed: 9'], axis = 1, inplace = True)

replace_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_space.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = bad_symbols.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text 

dataset['Story'] = dataset['Story'].apply(preprocess)
test['Story'] = test['Story'].apply(preprocess)



In [0]:
DATA_PATH = Path('data/')
DATA_PATH.mkdir(exist_ok=True)

LABEL_PATH = Path('label/')
LABEL_PATH.mkdir(exist_ok=True)

MODEL_PATH= Path('models/')
MODEL_PATH.mkdir(exist_ok=True)

OUTPUT_DIR = MODEL_PATH/'output'
OUTPUT_DIR.mkdir(exist_ok=True)

sub = pd.read_csv('sample_submission.csv', index_col=None)

print(dataset.shape, test.shape, sub.shape)

section_ohe = pd.get_dummies(dataset['Patient_Tag'])
train = pd.concat((dataset, section_ohe), axis=1)

from sklearn.model_selection import train_test_split

test_pct = np.round(len(test)/len(train),2)
train_1, valid_1 = train_test_split(train, test_size=test_pct, random_state=424242, stratify=train['Patient_Tag'])

print(train_1.shape, valid_1.shape)

train_1.to_csv(DATA_PATH/"train.csv", index=None)
valid_1.to_csv(DATA_PATH/"valid.csv", index=None)

test.to_csv("data/test.csv", index=None)

labels = pd.Series([0, 1])
labels.to_csv("label/labels.csv", index=None)

print("data saved successfully")

del train, section_ohe, train_1, valid_1
import gc; gc.collect()

In [0]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='xlnet-base-cased',
                          train_file='train.csv',
                          val_file='valid.csv',
                          label_file='labels.csv',
                          text_col='Story',
                          label_col=['0','1'],
                          batch_size_per_gpu=24,
                          max_seq_length=256,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='xlnet')

torch.cuda.empty_cache()
import gc; gc.collect()

In [0]:
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='xlnet-base-cased',#xlnet-large-cased, bert-base-uncased
                        metrics=metrics,
                        device=device_cuda,
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=None,
                        warmup_steps=500,
                        multi_gpu=False,
                        is_fp16=True,
                        multi_label=False,
                        logging_steps=50)

In [9]:
!pip install apex

In [4]:
#learner.save_model()
learner.fit(epochs=6,
            lr=1e-5,
            validate=True, 	# Evaluate the model after each epoch
            schedule_type="warmup_cosine")

learner.save_model()

MODEL_PATH_FILE = OUTPUT_DIR/'model_out'

predictor = BertClassificationPredictor(
                model_path=MODEL_PATH_FILE,
                label_path=LABEL_PATH, # location for labels.csv file
                multi_label=False,
                model_type='xlnet',
                do_lower_case=True)


texts = list(test['Story'])
pred = []
pred_probs = []
for text in tqdm(texts):
    result = predictor.predict(text)
    pred.append(int(result[0][0]))
    pred_probs.append([prob for label, prob in sorted(result, key=lambda x:x[0])])

print("test predictions")
print(pd.Series(pred).value_counts())

submission = pd.DataFrame()
submission['Index'] = index
submission['Patient_Tag'] = pd.Series(pred).astype(int)
submission.to_csv("model_xlnet_base.csv", index=None)

np.save("model_xlnet_base_prob.npy", np.array(pred_probs))

from tqdm import tqdm_notebook as tqdm
valid = pd.read_csv('data/valid.csv', index_col=None)

valid_pred = []
texts = list(valid['Story'])
for text in tqdm(texts):
    valid_pred.append(predictor.predict(text)[0][0])

from sklearn.metrics import accuracy_score
print(accuracy_score(valid['Patient_Tag'].values, pd.Series(valid_pred).astype(int)))

NameError: ignored